# Create new Schema final_exam

In [1]:
import sqlalchemy as sa
import pandas as pd
import requests, json

In [2]:
my_db_secret = {
    'drivername': 'postgresql+psycopg2', 
    'host': 'mmai5100postgres.canadacentral.cloudapp.azure.com',
    'port': '5432', 
    'username': 'shawnmd', 
    'password': '2023!Schulich',
    'database': 'shawnmd_db'
}

In [3]:
my_db_url = sa.engine.URL.create(
    drivername = my_db_secret['drivername'],
    username = my_db_secret['username'],
    password = my_db_secret['password'],
    host = my_db_secret['host'],
    port = my_db_secret['port'],
    database = my_db_secret['database']
)

In [4]:
my_db_engine = sa.create_engine(my_db_url)

In [6]:
with my_db_engine.connect() as connection:
        connection.execute(sa.text('CREATE SCHEMA IF NOT EXISTS final_exam;'))

# Get date_dimension table from mmai_db

In [7]:
mmai_db_secret = {
    'drivername' : 'postgresql+psycopg2',
    'host'       : 'mmai5100postgres.canadacentral.cloudapp.azure.com',
    'port'       : '5432',
    'username'   : 'shawnmd',
    'password'   : '2023!Schulich',
    'database'   : 'mmai_db'
}

In [8]:
mmai_db_url = sa.engine.URL.create(
    drivername = mmai_db_secret['drivername'] ,
    username   = mmai_db_secret['username'],
    password   = mmai_db_secret['password'] ,
    host       = mmai_db_secret['host'] ,
    port       = mmai_db_secret['port'] ,
    database   = mmai_db_secret['database']
)

In [9]:
mmai_db_engine = sa.create_engine(mmai_db_url)

In [13]:
with mmai_db_engine.connect() as connection:
    date_data = pd.read_sql(
        sql = sa.text('SELECT * FROM dimensions.date_dimension'),
        con = connection
    )

date_data

,sk_date,date,day_name,day_of_month,day_of_year,month,month_name,year,year_week,week,running_week,year_quarter,quarter,running_quarter
0,20190101,2019-01-01,Tuesday,1,1,1,January,2019,2019-W01,W01,1,2019-Q1,Q1,1
1,20190102,2019-01-02,Wednesday,2,2,1,January,2019,2019-W01,W01,1,2019-Q1,Q1,1
2,20190103,2019-01-03,Thursday,3,3,1,January,2019,2019-W01,W01,1,2019-Q1,Q1,1
3,20190104,2019-01-04,Friday,4,4,1,January,2019,2019-W01,W01,1,2019-Q1,Q1,1
4,20190105,2019-01-05,Saturday,5,5,1,January,2019,2019-W01,W01,1,2019-Q1,Q1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,20231227,2023-12-27,Wednesday,27,361,12,December,2023,2023-W52,W52,261,2023-Q4,Q4,20
1822,20231228,2023-12-28,Thursday,28,362,12,December,2023,2023-W52,W52,261,2023-Q4,Q4,20
1823,20231229,2023-12-29,Friday,29,363,12,December,2023,2023-W52,W52,261,2023-Q4,Q4,20
1824,20231230,2023-12-30,Saturday,30,364,12,December,2023,2023-W52,W52,261,2023-Q4,Q4,20


In [14]:
date_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1826 entries, 0 to 1825
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   sk_date          1826 non-null   int64 
 1   date             1826 non-null   object
 2   day_name         1826 non-null   object
 3   day_of_month     1826 non-null   int64 
 4   day_of_year      1826 non-null   int64 
 5   month            1826 non-null   int64 
 6   month_name       1826 non-null   object
 7   year             1826 non-null   int64 
 8   year_week        1826 non-null   object
 9   week             1826 non-null   object
 10  running_week     1826 non-null   int64 
 11  year_quarter     1826 non-null   object
 12  quarter          1826 non-null   object
 13  running_quarter  1826 non-null   int64 
dtypes: int64(7), object(7)
memory usage: 199.8+ KB


# Input the date_data into final_exam schema

In [16]:
date_data.to_sql(
    name = 'date_dimension',
    con = my_db_engine,
    schema = 'final_exam',
    if_exists = 'replace',
    index = False,
    method = 'multi',
    dtype = {
        'sk_date ': sa.types.INTEGER,
        'date': sa.types.VARCHAR(100),
        'day_name ': sa.types.VARCHAR(100),
        'day_of_month' : sa.types.INTEGER, 
        'day_of_year': sa.types.INTEGER,
        'month': sa.types.INTEGER,
        'month_name': sa.types.VARCHAR(100),
        'year': sa.types.INTEGER, 
        'year_week':sa.types.VARCHAR(100),
        'week': sa.types.VARCHAR(100),
        'running_week': sa.types.INTEGER,
        'year_quarter':sa.types.VARCHAR(100),
        'quarter':sa.types.VARCHAR(100),
        'running_quarter': sa.types.INTEGER,
    }
)

1826